## 번역 모델 만들기
실습에선 접근성이 좋은 영어-스페인어 데이터를 사용하도록 하겠습니다.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.4.1


In [4]:
path_to_zip = tf.keras.utils.get_file(
    'spa_eng_zip',
    origin = 'http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip) + "/spa-eng/spa.txt"

with open(path_to_file, "r") as f:
    corpus = f.read().splitlines()
    
print("Data Size:", len(corpus))
print("Example:")

for sen in corpus[0:100][::20]: print(">>", sen)

Data Size: 118964
Example:
>> Go.	Ve.
>> Wait.	Esperen.
>> Hug me.	Abrázame.
>> No way!	¡Ni cagando!
>> Call me.	Llamame.


두 언어가 단어 사전을 공유하도록 하겠습니다. 영어와 스페인어 모두 알파벳으로 이뤄지는 데다가 같은 인도유럽어족이기 때문에 기대할 수 있는 효과가 많아요! 후에 챗봇을 만들 때에도 질문과 답변이 모두 한글로 이루어져 있기 때문에 Embedding 층을 공유하는 것이 성능에 도움이 됩니다.  
  
토큰화에는 Sentencepiece를 사용할 것이고 단어 사전 수는 20,000으로 설정하겠습니다.

### 토큰화
중복데이터를 set 데이터형을 활용해 제거한 후 Sentencepiece 기반의 토크나이저를 생성해 주는 gernerate_tokenizer() 함수를 정의하여 토크나이저를 얻습니다.

In [14]:
def generate_tokenizer(corpus, vocab_size, lang="spa-eng",
                       pad_id=0, # pad toked의 일련번호
                       bos_id=1, # 문장의 시작을 의미하는 bos token(<s>)의 일련번호
                       eos_id=2, # 문장의 끝을 의미하는 eos token(<\s>)의 일련번호
                       unk_id=3): # unk token의 일련번호
    file = "./%s_corpus.txt"% lang
    model = "%s_spm"% lang
    
    with open(file, 'w') as f: # 위에서 지정한 file 경로에 저장합니다.
        for row in corpus: f.write(str(row) + '\n') # corpus를 열어 줄 단위로 읽습니다.
            
    import sentencepiece as spm
    spm.SentencePieceTrainer.Train(
        '--input=./%s --model_prefix=%s --vocab_size=%d'\
        % (file, model, vocab_size) + \
        '--pad_id==%d --bos_id=%d --eos_id=%d --unk_id=%d'\
        % (pad_id, bos_id, eos_id, unk_id)
    )
    
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load('%s.model' % model)
    
    return tokenizer        

In [15]:
cleaned_corpus = list(set(corpus))

VOCAB_SIZE = 20000
tokenizer = generate_tokenizer(cleaned_corpus, VOCAB_SIZE)
tokenizer.set_encode_extra_options("bos:eos") # 문장 양 끝에 <s>, </s> 추가

True

위에서 두 언어 사이에 단어 사전을 공유하기로 하였으므로, 따라서 Encoder와 Decoder의 전용 토크나이저를 만들지 않고, 방금 만들어진 토크나이저를 두 언어 사이에서 공유하게 됩니다. 
  
토크나이저가 준비되었으니 본격적으로 데이터를 토큰화하도록 하겠습니다. 문장부호와 대소문자 등을 정제하는 preprocess_sentence() 함수를 정의해 데이터를 정제하고, 정제된 데이터가 50개 이상의 토큰을 갖는 경우 제거하도록 합니다!

In [16]:
def preprocess_sentence(sentence):
    sentence = sentence.lower()
    
    sentence = re.sub(r"([?.!,¿¡])", r" \1 ", sentence) # [!?,.¿¡]를 그룹화 해준다. r"\1"은 첫 번째 그룹, 즉 [!?,.¿¡]를 가리킨다.
    sentence = re.sub(r'[" "]+', " ", sentence) # 띄어쓰기가 여러 번 나올 수 있는데 그걸 하나의 " "로 바꿔준다.
    sentence = re.sub(r"[^a-zA-Z?.!,¿¡]+", " ",sentence) # 대괄호[] 안에 있는 문자들이 여러번 나오는 경우를 제외하고 모두 " "로 대체한다.
    
    sentence = sentence.strip() # 스페이스 기준으로 분리
    
    return sentence

In [17]:
from tqdm import tqdm_notebook # Process(진행과정)을 보여주는 함수. 감싸기만 하면 된다

src_corpus = []
tgt_corpus = []

for pair in tqdm_notebook(cleaned_corpus):
    # pair는 eng-spa가 tab을 기준으로 분리된 상태. 따라서 tab을 기준으로 분리해준다.
    src, tgt = pair.split('\t')
    
    src_tokens = tokenizer.encode_as_ids(preprocess_sentence(src)) # encode_as_ids()는 문자열을 숫자로 분할합니다.
    tgt_tokens = tokenizer.encode_as_ids(preprocess_sentence(tgt))
    
    if (len(src_tokens) > 50): continue
    if (len(tgt_tokens) > 50): continue
        
    src_corpus.append(src_tokens)
    tgt_corpus.append(tgt_tokens)
    
len(src_corpus)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/118964 [00:00<?, ?it/s]

118951

pad_sequences()를 이용해 패딩을 진행하겠습니다. 데이터의 1%는 테스트 셋으로 빼놓겠습니다.

In [23]:
enc_tensor = tf.keras.preprocessing.sequence.pad_sequences(src_corpus, padding='post')
dec_tensor = tf.keras.preprocessing.sequence.pad_sequences(tgt_corpus, padding='post')

enc_train, enc_val, dec_train, dec_val = train_test_split(enc_tensor, dec_tensor, test_size=0.01)

print("enc_train:", len(enc_train), "enc_val:", len(enc_val))
print("dec_train:", len(dec_train), "dec_val:", len(dec_val))

enc_train: 117761 enc_val: 1190
dec_train: 117761 dec_val: 1190


### 트랜스포머 구현하기
Encoder와 Decoder 각각의 Embedding과 출력층의 Linear, 총 3개의 레이어가 Weight를 공유할 수 있게 하세요

**Positional Encoding**  
$PE(pos, 2i) = Sin(pos/10000^{\frac{2i}{d_{model}}})$  
$PE(pos, 2i+1) = Cos(pos/10000^{\frac{2i+1}{d_{model}}})$

In [24]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)
    
    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]
    
    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])
    
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
    
    return sinusoid_table

**Generate Padding Mask**

In [25]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)
    
    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)
    
    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)
    
    return enc_mask, dec_enc_mask, dec_mask

**Multi-head Attention**
$Attention(Q, K, V) = Softmax(\frac{QK^T}{\sqrt{d_k}})V$

In [26]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.kreas.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)
        
        scaled_qk = QK / tf.math.sqrt(d_k)
        
        if mask is not None: scaled_qk += (mask * -1e9)
            
        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)
        
        return out, attentions
    
    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])
        
        return split_x
    
    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))
        
        return combined_x
    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
        
        out = self.combine_heads(out)
        out = self.linear(out)
        
        return out, attention_weights

**Position-wise Feed Forward Network**

In [27]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_moel
        self.d_ff = d_ff
        
        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)
        
    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        
        return out

**Encoder Layer**

In [28]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        
        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)
        
        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        """
        Position-Wise Feed Forward Netword
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

**Decoder Layer**